In [75]:
import berry_flux_diag as bfd
from pymatgen.core import Lattice, Structure
from pymatgen.io.vasp import Poscar
import numpy as np

In [66]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Notes: This shows an example for a non-spin polarized calculation without DFT+U. I have working examples to support both spin-polarized calculations and DFT+U, and eventually I will have tutorials covering how to pre-process input files for these kinds of calculations, so please contact poteshman@uchicago.edu for these working examples while the tutorial for these cases are in progress.

### Step 0: Have two reference structures (e.g., polar and nonpolar) that have already been relaxed and have the same atoms in the same order 

### Step 1: Set up two reference structures (e.g., polar and nonpolar) as pymatgen Structure objects

In [67]:
material_name = 'BaTiO3'
species = ["Ba", "Ti", "O", "O", "O"] # order must match coordinates

In [68]:
# nonpolar BaTiO3 structure
np_lattice = Lattice([
    [4.0355829999999999, 0.0, 0.0],
    [0.0, 4.0355829999999999, 0.0],
    [0.0, 0.0, 4.0355829999999999]
])

np_coords = [
    [0.0000000000000000, 0.0000000000000000, 0.0028000000000000],  # Ba
    [0.5000000000000000, 0.5000000000000000, 0.5028000000000000],  # Ti
    [0.0000000000000000, 0.5000000000000000, 0.5028000000000000],  # O
    [0.5000000000000000, 0.0000000000000000, 0.5028000000000000],  # O
    [0.5000000000000000, 0.5000000000000000, 0.0028000000000000],  # O
]

np_orig_struct = Structure(lattice=np_lattice, species=species, coords=np_coords)

In [69]:
# polar BaTiO3 structure
pol_lattice = Lattice([
    [4.0020059999999997, 0.0, 0.0],
    [0.0, 4.0020059999999997, 0.0],
    [0.0, 0.0, 4.2155069999999997]
])

pol_coords = [
    [0.0, 0.0, 0.0202290000000000],  # Ba
    [0.5, 0.5, 0.5389210000000000],  # Ti
    [0.0, 0.5, 0.4920230000000000],  # O
    [0.5, 0.0, 0.4920230000000000],  # O
    [0.5, 0.5, 0.9708030000000000],  # O
]

pol_orig_struct = Structure(lattice=pol_lattice, species=species, coords=pol_coords)

### Step 2: Preprocess structs to apply proper translation and generate interpolated structures if necessary

In [70]:
structs_dict = bfd.preprocess.preprocess_structs(pol_orig_struct, np_orig_struct,
                                                translate=True, # apply translation that minimizes maximal atomic displacement
                                                num_interps='auto', # generate the number of interpolated structures necessary 
                                                MAX_DISP=0.3) # max atomic displacement between any 2 correspond atoms in structs (Angstroms)
structs = structs_dict["structs"]

### Step 3a (QE): Write scf.in files for each reference structure (with properly-applied translation between cells and number of interpolated structures automatically calculated)

In [71]:
save_path = 'QE_io_files/'
material = 'BaTiO3'
pseudo_dir = '/global/homes/p/pabigail/qe_pseudos/' # need ONCV pseudopotentials
kpoints_grid = (5, 5, 5) # k-point mesh
kshift = (0, 0, 0)
sym = False # need to compute wavefunctions over entire BZ (not IBZ)
species_dict = {"species": ["Ba", "O", "Ti"],
                "masses": [137.3270, 15.9994, 47.8670],
                "pseudos": ["Ba_ONCV_PBE-1.0.upf",
                            "O_ONCV_PBE-1.0.upf",
                            "Ti_ONCV_PBE-1.0.upf"]}

bfd.preprocess.write_qe_in_scf_files(save_path, structs, material,
                                     pseudo_dir, kpoints_grid, kshift, species_dict)

0

### Step 3b (VASP): Write INCAR & POSCAR files

In [77]:
save_path = 'VASP_io_files/'
material = 'BaTiO3'

# write POSCAR files
bfd.preprocess.write_vasp_in_poscar_files(save_path, structs, material)

Wrote VASP_io_files/POSCAR_BaTiO3_pol_orig
Wrote VASP_io_files/POSCAR_BaTiO3_np_trans


`INCAR` files:
set 
```bash
ISYM = -1
LWAVE = True
```
and for `KPOINT` files, ensure that all calculations 

Reminder: Enforce the same k-point discretization for all structures